# Predicting Ambulance Call Outs on a Specific Date.

In [29]:
import pandas as pd
from patsy import dmatrices
import numpy as np
import statsmodels.api as sm
import matplotlib.pyplot as plt
import statsmodels.formula.api as smf
from sklearn.metrics import r2_score
def mean_absolute_percentage_error(y_true, y_pred): 
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

In [30]:
df = pd.read_csv('AmbulanceCallOuts.csv', header=0, infer_datetime_format=True, parse_dates=[0], index_col=[0])
df["EVENT"] = df["EVENT"].fillna("No Event")
df['MONTH'] = df['MONTH'].astype(object)
df['DAY_OF_WEEK'] = df['DAY_OF_WEEK'].astype(object)
df['DAY'] = df['DAY'].astype(object)
df['HOUR'] = df['HOUR'].astype(object)

## Baseline - Poisson With Time Features

In [31]:
mask = np.random.rand(len(df)) < 0.7
df_train = df[mask]
df_test = df[~mask]
print('Train Set ='+str(len(df_train)))
print('Test Set ='+str(len(df_test)))
expr = """COUNT ~ DAY_OF_WEEK + MONTH + DAY + HOUR + CLINICAL_STATUS + STATION_AREA"""
y_train, X_train = dmatrices(expr, df_train, return_type='dataframe')
y_test, X_test = dmatrices(expr, df_test, return_type='dataframe')
while X_train.shape[1] != X_test.shape[1]:
    mask = np.random.rand(len(df)) < 0.7
    df_train = df[mask]
    df_test = df[~mask]
    y_train, X_train = dmatrices(expr, df_train, return_type='dataframe')
    y_test, X_test = dmatrices(expr, df_test, return_type='dataframe')
print(X_train.shape)
print(X_test.shape)

Train Set =101534
Test Set =43892
(101534, 92)
(43892, 92)


In [32]:
poisson_training_results1 = sm.GLM(y_train, X_train, family=sm.families.Poisson()).fit()
print(poisson_training_results1.summary())

                 Generalized Linear Model Regression Results                  
Dep. Variable:                  COUNT   No. Observations:               101534
Model:                            GLM   Df Residuals:                   101442
Model Family:                 Poisson   Df Model:                           91
Link Function:                    log   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:            -4.3970e+05
Date:                Mon, 11 May 2020   Deviance:                   1.5457e+05
Time:                        16:06:58   Pearson chi2:                 1.57e+05
No. Iterations:                     4                                         
Covariance Type:            nonrobust                                         
                                     coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------------------------
Intercept   

In [33]:
poisson_predictions1 = poisson_training_results1.get_prediction(X_test)
predictions_summary_frame1 = poisson_predictions1.summary_frame()

In [34]:
poisson1mean = predictions_summary_frame1["mean"]

## Negative Binomial Model #1 - With Time Features

In [35]:
expr = """COUNT ~ DAY_OF_WEEK + MONTH + DAY + HOUR + CLINICAL_STATUS"""
y_train, X_train = dmatrices(expr, df_train, return_type='dataframe')
y_test, X_test = dmatrices(expr, df_test, return_type='dataframe')

In [36]:
poisson_training_results = sm.GLM(y_train, X_train, family=sm.families.Poisson()).fit()
print(poisson_training_results.mu)
df_train['LAMBDA'] = poisson_training_results.mu
df_train['AUX_OLS_DEP'] = df_train.apply(lambda x: ((x['COUNT'] - x['LAMBDA'])**2 - x['COUNT']) / x['LAMBDA'], axis=1)

[199.77299164 198.17491686 198.44670172 ... 209.15869731 209.1637236
 209.01782554]


C:\Users\Matt\anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\Matt\anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [37]:
ols_expr = """AUX_OLS_DEP ~ LAMBDA - 1"""
aux_olsr_results = smf.ols(ols_expr, df_train).fit()
print(aux_olsr_results.params)
aux_olsr_results.tvalues

LAMBDA    0.002765
dtype: float64


LAMBDA    50.875278
dtype: float64

In [38]:
nb2_training_results0 = sm.GLM(y_train, X_train,family=sm.families.NegativeBinomial(alpha=aux_olsr_results.params[0])).fit()
print(nb2_training_results0.summary())

                 Generalized Linear Model Regression Results                  
Dep. Variable:                  COUNT   No. Observations:               101534
Model:                            GLM   Df Residuals:                   101457
Model Family:        NegativeBinomial   Df Model:                           76
Link Function:                    log   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:            -4.3414e+05
Date:                Mon, 11 May 2020   Deviance:                       98548.
Time:                        16:07:18   Pearson chi2:                 1.00e+05
No. Iterations:                     5                                         
Covariance Type:            nonrobust                                         
                                     coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------------------------
Intercept   

In [39]:
nb2_predictions0 = nb2_training_results0.get_prediction(X_test)
nb2_predictions_summary_frame0 = nb2_predictions0.summary_frame()

In [40]:
nb0mean = nb2_predictions_summary_frame0["mean"]

## Negative Binomial Model #2 - With Weather Features

In [41]:
expr = """COUNT ~ DAY_OF_WEEK + MONTH + DAY + TEMPERATURE + CLOUD_COVER + PRECIPITATION + HOUR + CLINICAL_STATUS"""
y_train, X_train = dmatrices(expr, df_train, return_type='dataframe')
y_test, X_test = dmatrices(expr, df_test, return_type='dataframe')

In [42]:
poisson_training_results = sm.GLM(y_train, X_train, family=sm.families.Poisson()).fit()
print(poisson_training_results.mu)
df_train['LAMBDA'] = poisson_training_results.mu
df_train['AUX_OLS_DEP'] = df_train.apply(lambda x: ((x['COUNT'] - x['LAMBDA'])**2 - x['COUNT']) / x['LAMBDA'], axis=1)

[200.18102507 198.3187109  198.68923036 ... 211.10877686 211.0177769
 210.97713172]


C:\Users\Matt\anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\Matt\anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [43]:
ols_expr = """AUX_OLS_DEP ~ LAMBDA - 1"""
aux_olsr_results = smf.ols(ols_expr, df_train).fit()

In [44]:
nb2_training_results1 = sm.GLM(y_train, X_train,family=sm.families.NegativeBinomial(alpha=aux_olsr_results.params[0])).fit()
print(nb2_training_results1.summary())

                 Generalized Linear Model Regression Results                  
Dep. Variable:                  COUNT   No. Observations:               101534
Model:                            GLM   Df Residuals:                   101454
Model Family:        NegativeBinomial   Df Model:                           79
Link Function:                    log   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:            -4.3311e+05
Date:                Mon, 11 May 2020   Deviance:                       98706.
Time:                        16:07:39   Pearson chi2:                 1.01e+05
No. Iterations:                     5                                         
Covariance Type:            nonrobust                                         
                                     coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------------------------
Intercept   

In [45]:
nb2_predictions1 = nb2_training_results1.get_prediction(X_test)
nb2_predictions_summary_frame1 = nb2_predictions1.summary_frame()

In [46]:
nb1mean = nb2_predictions_summary_frame1["mean"]

## Negative Binomial Model #3 - With Event Features

In [47]:
expr = """COUNT ~ DAY_OF_WEEK + MONTH + DAY + EVENT + HOUR + CLINICAL_STATUS"""
y_train, X_train = dmatrices(expr, df_train, return_type='dataframe')
y_test, X_test = dmatrices(expr, df_test, return_type='dataframe')

In [48]:
poisson_training_results = sm.GLM(y_train, X_train, family=sm.families.Poisson()).fit()
print(poisson_training_results.mu)
df_train['LAMBDA'] = poisson_training_results.mu
df_train['AUX_OLS_DEP'] = df_train.apply(lambda x: ((x['COUNT'] - x['LAMBDA'])**2 - x['COUNT']) / x['LAMBDA'], axis=1)

[200.88645323 199.22626807 199.51180005 ... 208.93140157 208.90891333
 208.80036843]


C:\Users\Matt\anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\Matt\anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [49]:
ols_expr = """AUX_OLS_DEP ~ LAMBDA - 1"""
aux_olsr_results = smf.ols(ols_expr, df_train).fit()

In [50]:
nb2_training_results2 = sm.GLM(y_train, X_train,family=sm.families.NegativeBinomial(alpha=aux_olsr_results.params[0])).fit()
print(nb2_training_results2.summary())

                 Generalized Linear Model Regression Results                  
Dep. Variable:                  COUNT   No. Observations:               101534
Model:                            GLM   Df Residuals:                   101453
Model Family:        NegativeBinomial   Df Model:                           80
Link Function:                    log   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:            -4.3388e+05
Date:                Mon, 11 May 2020   Deviance:                       98530.
Time:                        16:08:03   Pearson chi2:                 1.00e+05
No. Iterations:                     5                                         
Covariance Type:            nonrobust                                         
                                     coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------------------------
Intercept   

In [51]:
nb2_predictions2 = nb2_training_results2.get_prediction(X_test)
nb2_predictions_summary_frame2 = nb2_predictions2.summary_frame()

In [52]:
nb2mean = nb2_predictions_summary_frame2["mean"]

## Evaluation

In [53]:
X_test["ACTUAL"] = y_test["COUNT"]
actual = X_test["ACTUAL"]
X_test["POISSON1_PREDICTIONS"] = poisson1mean
X_test["NB0_PREDICTIONS"] = nb0mean
X_test["NB1_PREDICTIONS"] = nb1mean
X_test["NB2_PREDICTIONS"] = nb2mean

In [54]:
poisson1R2 = r2_score(actual, poisson1mean)
nb0R2 = r2_score(actual, nb0mean)
nb1R2 = r2_score(actual, nb1mean)
nb2R2 = r2_score(actual, nb2mean)

print("R2 Score for Poisson Model with Time Features              =  {:.4f}".format(poisson1R2))
print("R2 Score for Negative Binomial Model with Time Features    =  {:.4f}".format(nb0R2))
print("R2 Score for Negative Binomial Model with Weather Features =  {:.4f}".format(nb1R2))
print("R2 Score for Negative Binomial Model with Event Features   =  {:.4f}".format(nb2R2))

R2 Score for Poisson Model with Time Features              =  0.3440
R2 Score for Negative Binomial Model with Time Features    =  0.3438
R2 Score for Negative Binomial Model with Weather Features =  0.3588
R2 Score for Negative Binomial Model with Event Features   =  0.3472


In [55]:
poisson1MAE = mean_absolute_percentage_error(actual, poisson1mean)
nb0MAE = mean_absolute_percentage_error(actual, nb0mean)
nb1MAE = mean_absolute_percentage_error(actual, nb1mean)
nb2MAE = mean_absolute_percentage_error(actual, nb2mean)

print("Mean Absolute Percentage Error for Poisson Model with Time Features              = {:.4f}".format(poisson1MAE))
print("Mean Absolute Percentage Error for Negative Binomial Model with Time Features    = {:.4f}".format(nb0MAE))
print("Mean Absolute Percentage Error for Negative Binomial Model with Weather Features = {:.4f}".format(nb1MAE))
print("Mean Absolute Percentage Error for Negative Binomial Model with Event Features   = {:.4f}".format(nb2MAE))

Mean Absolute Percentage Error for Poisson Model with Time Features              = 6.6922
Mean Absolute Percentage Error for Negative Binomial Model with Time Features    = 6.6922
Mean Absolute Percentage Error for Negative Binomial Model with Weather Features = 6.5620
Mean Absolute Percentage Error for Negative Binomial Model with Event Features   = 6.6698
